<a href="https://colab.research.google.com/github/vidyabandgar97/A2A/blob/main/detectron2_500_30_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/detectron2_500/

/content/drive/MyDrive/detectron2_500


In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
     |████████████████████████████████| 703.8 MB 23 kB/s 
     |████████████████████████████████| 6.6 MB 40.9 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.5.0+cu101 which is incompatible.
     |████████████████████████████████| 274 kB 5.4 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=1bc3020ef0a4bdcd72a9fbab7cd50b

In [ ]:
!pip install torch==1.9.0 torchvision==0.4.1

     |████████████████████████████████| 831.4 MB 2.7 kB/s 
     |████████████████████████████████| 10.1 MB 10.4 MB/s 
INFO: pip is looking at multiple versions of <Python from Requires-Python> to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install torch==1.9.0 and torchvision==0.4.1 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested torch==1.9.0
    torchvision 0.4.1 depends on torch==1.3.0

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip attempt to solve the dependency conflict

ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/user_guide/#fixing-conflicting-dependencies


In [ ]:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html
     |████████████████████████████████| 6.2 MB 753 kB/s 
     |████████████████████████████████| 49 kB 4.6 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20210825-py3-none-any.whl size=60661 sha256=e0b9efde31c744e4dbcd687bdf3579731dbca8048d03183f820cf8ebaed83507
  Stored in directory: /root/.cache/pip/wheels/53/c4/f8/c4cb07f135845218b019b4a55d8a0470a0f21ee13f8dcd16be
Successfully built fvcore


In [ ]:
# check pytorch installation: 
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
#assert torch.__version__.startswith("1.9")   # please manually install torch 1.9 if Colab changes its default version

1.5.0+cu101 True


In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer

In [ ]:
# if your dataset is in COCO format, this cell can be replaced by the following three lines:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train2", {}, "/content/drive/MyDrive/detectron2_500/train.json", "/content/drive/MyDrive/detectron2_500/Dataset")
#register_coco_instances("my_dataset_val", {}, "/content/drive/MyDrive/detectron_train/test/test.json", "/content/drive/MyDrive/detectron_train/test")

In [ ]:
from detectron2.data.datasets import register_coco_instances
#register_coco_instances("my_dataset_train2", {}, "/content/drive/MyDrive/detectron2_500/train.json", "/content/drive/MyDrive/detectron2_500/Dataset")
register_coco_instances("my_dataset_val2", {}, "/content/drive/MyDrive/detectron2_500/test.json", "/content/drive/MyDrive/detectron2_500/test_data")

In [ ]:
register_coco_instances("my_dataset_test2", {}, "/content/drive/MyDrive/detectron2_500/test.json", "/content/drive/MyDrive/detectron2_500/test_data")

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
import os

cfg = get_cfg()
cfg.OUTPUT_DIR = "/content/drive/MyDrive/detectron2_500/output_detect"
cfg.SOLVER.CHECKPOINT_PERIOD = 100
cfg.merge_from_file(model_zoo.get_config_file("Misc/mask_rcnn_R_50_FPN_3x_dconv_c3-c5.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train2",)
#cfg.DATASETS.TEST = ("my_dataset_train",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("Misc/mask_rcnn_R_50_FPN_3x_dconv_c3-c5.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 35000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 6  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=True)
trainer.train()
#trainer.test()


[08/30 06:58:28 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

FileNotFoundError: ignored

In [ ]:
import os
#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/detectron2_500/output_detect/model_final.pth'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.25
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_val2", cfg, False, output_dir="/content/drive/MyDrive/detectron2_500/output_detect/")
val_loader = build_detection_test_loader(cfg, "my_dataset_val2")
inference_on_dataset(trainer.model, val_loader, evaluator)

[08/30 09:19:13 d2.data.datasets.coco]: Loaded 190 images in COCO format from /content/drive/MyDrive/detectron2_500/test.json
[08/30 09:19:13 d2.data.common]: Serializing 190 elements to byte tensors and concatenating them all ...
[08/30 09:19:13 d2.data.common]: Serialized dataset takes 0.34 MiB
[08/30 09:19:13 d2.evaluation.evaluator]: Start inference on 190 images
[08/30 09:19:15 d2.evaluation.evaluator]: Inference done 11/190. 0.1138 s / img. ETA=0:00:21
[08/30 09:19:20 d2.evaluation.evaluator]: Inference done 50/190. 0.1235 s / img. ETA=0:00:18
[08/30 09:19:25 d2.evaluation.evaluator]: Inference done 89/190. 0.1243 s / img. ETA=0:00:13
[08/30 09:19:30 d2.evaluation.evaluator]: Inference done 127/190. 0.1246 s / img. ETA=0:00:08
[08/30 09:19:35 d2.evaluation.evaluator]: Inference done 165/190. 0.1248 s / img. ETA=0:00:03
[08/30 09:19:38 d2.evaluation.evaluator]: Total inference time: 0:00:24.405979 (0.131924 s / img per device, on 1 devices)
[08/30 09:19:39 d2.evaluation.evaluator]

OrderedDict([('bbox',
              {'AP': 31.41083263855518,
               'AP-aeroplane': 54.23060243914306,
               'AP-car': 16.344716890771043,
               'AP-chair': 23.05291033820693,
               'AP-cow': 54.62034935645023,
               'AP-person': 13.890689693915217,
               'AP-traffic_light': 26.325727112844643,
               'AP50': 47.526916227092265,
               'AP75': 32.765843706091175,
               'APl': 48.35890565298943,
               'APm': 42.14435574933468,
               'APs': 16.090924820200165}),
             ('segm',
              {'AP': 25.026855619500676,
               'AP-aeroplane': 38.28955295050819,
               'AP-car': 14.907862600953889,
               'AP-chair': 17.019894750559573,
               'AP-cow': 45.20911096876206,
               'AP-person': 11.282582838677053,
               'AP-traffic_light': 23.45212960754331,
               'AP50': 44.34470655886626,
               'AP75': 27.188828113352447,
  

In [ ]:
import os
#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/detectron2_500/output_detect/model_final.pth'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.25
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_train2", cfg, False, output_dir="/content/drive/MyDrive/detectron2_500/output_detect/")
val_loader = build_detection_test_loader(cfg, "my_dataset_train2")
inference_on_dataset(trainer.model, val_loader, evaluator)

[08/30 09:13:24 d2.data.datasets.coco]: Loaded 689 images in COCO format from /content/drive/MyDrive/detectron2_500/train.json
[08/30 09:13:24 d2.data.common]: Serializing 689 elements to byte tensors and concatenating them all ...
[08/30 09:13:24 d2.data.common]: Serialized dataset takes 1.35 MiB
[08/30 09:13:24 d2.evaluation.evaluator]: Start inference on 689 images
[08/30 09:13:25 d2.evaluation.evaluator]: Inference done 11/689. 0.1214 s / img. ETA=0:01:25
[08/30 09:13:30 d2.evaluation.evaluator]: Inference done 52/689. 0.1177 s / img. ETA=0:01:18
[08/30 09:13:36 d2.evaluation.evaluator]: Inference done 91/689. 0.1194 s / img. ETA=0:01:15
[08/30 09:13:41 d2.evaluation.evaluator]: Inference done 131/689. 0.1195 s / img. ETA=0:01:10
[08/30 09:13:46 d2.evaluation.evaluator]: Inference done 171/689. 0.1201 s / img. ETA=0:01:05
[08/30 09:13:51 d2.evaluation.evaluator]: Inference done 211/689. 0.1200 s / img. ETA=0:01:00
[08/30 09:13:56 d2.evaluation.evaluator]: Inference done 251/689. 0.

OrderedDict([('bbox',
              {'AP': 83.98985083516854,
               'AP-aeroplane': 94.4253694485331,
               'AP-car': 81.1134626599985,
               'AP-chair': 84.26749502289809,
               'AP-cow': 85.60505749591054,
               'AP-person': 83.23153904779109,
               'AP-traffic_light': 75.29618133587996,
               'AP50': 97.98701363695378,
               'AP75': 94.24580352017277,
               'APl': 93.8941785222392,
               'APm': 89.71633810740357,
               'APs': 78.86993842269797}),
             ('segm',
              {'AP': 64.61726225944165,
               'AP-aeroplane': 67.15611327462017,
               'AP-car': 67.1606684741333,
               'AP-chair': 60.98327678821849,
               'AP-cow': 63.3536607440159,
               'AP-person': 62.74180126421302,
               'AP-traffic_light': 66.308053011449,
               'AP50': 95.80969673089574,
               'AP75': 77.09854638785755,
               'APl'

In [ ]:
!test_inference.py Misc/mask_rcnn_R_50_FPN_3x_dconv_c3-c5.yaml /content/drive/MyDrive/detectron2_500/output_detect/model_0034999.pth --opts MODEL.WEIGHTS detectron2://COCO-Detection/faster_rcnn_R_50_FPN_3x/137849458//content/drive/MyDrive/detectron2_500/output_detect/model_final.pth